# Candidate Scoring System (1-100)

This notebook is designed to extract data from a SQLite database and calculate a score for candidates from 1 to 100 based on specific requirements.

In [ ]:
# Import necessary libraries
import sqlite3
import pandas as pd
import numpy as np


## Step 1: Connect to the SQLite Database and Extract Data

In [ ]:
# Connect to the database
db_path = '/mnt/data/teste___desafio_técnico_-_analista_de_dados.db'
conn = sqlite3.connect(db_path)

# Write your SQL query
query = '''
SELECT
    v.id AS vaga_id,
    v.nome AS nome_vaga,
    c.id AS candidato_id,
    c.nome AS nome_candidato,
    c.estado AS candidato_estado,
    cv.pretensao_salarial,
    v.salario_minimo,
    v.salario_maximo,
    ce.total_years AS anos_experiencia_total,
    v.minimo_experiencia,
    comp.nome AS competencia,
    ce.tempo_competencia / 365.25 AS anos_competencia,
    vc.tempo_de_experiencia,
    vc.nivel_competencia
FROM
    vagas v
JOIN
    vagacompetencia vc ON v.id = vc.id_vaga
JOIN
    competencias comp ON vc.id_competencia = comp.id
JOIN
    competencia_experiencia ce ON comp.id = ce.id_competencia
JOIN
    experiencias e ON ce.id_experiencia = e.id
JOIN
    candidatos c ON e.id_candidato = c.id
JOIN
    candidato_vaga cv ON cv.id_candidato = c.id AND cv.id_vaga = v.id
GROUP BY
    v.id, c.id;
'''

# Load data into a DataFrame
df = pd.read_sql_query(query, conn)
df.head()

## Step 2: Define the Scoring Criteria

In [ ]:
# Define the company states
company_states = ['SC', 'PE', 'SP']  # Santa Catarina, Pernambuco, São Paulo

# Define a function to calculate the score based on the criteria
def calculate_score(row):
    score = 0

    # Criterion 1: Location (25 points)
    if row['candidato_estado'] in company_states:  # State matches one of the company states
        score += 25

    # Criterion 2: Salary fit (25 points)
    mid_salary = (row['salario_minimo'] + row['salario_maximo']) / 2
    if row['salario_minimo'] < row['pretensao_salarial'] < row['salario_maximo']:
        # Closer to mid-salary gets a better score
        diff_to_mid = abs(mid_salary - row['pretensao_salarial'])
        max_diff = row['salario_maximo'] - row['salario_minimo']
        score += 25 * (1 - (diff_to_mid / max_diff))  # Normalize score

    # Criterion 3: Competence Fit (20 points)
    if row['nivel_competencia'] == 'Avançado':
        score += 20
    elif row['nivel_competencia'] == 'Intermediário':
        score += 10

    # Criterion 4: Experience (20 points)
    if row['anos_experiencia_total'] >= 5:  # More than 5 years experience
        score += 20

    return round(score, 2)  # Score out of 100

# Apply the scoring function
df['score'] = df.apply(calculate_score, axis=1)
df[['nome_candidato', 'nome_vaga', 'score']].head()

## Step 3: Save the Scored Data

In [ ]:
# Save the scored DataFrame to a CSV file
output_path = 'scored_candidates.csv'
df.to_csv(output_path, index=False)
print(f'Scored candidates saved to {output_path}')

This notebook can be rerun every 15 days to generate a new candidate score.